In [4]:
import numpy as np
import scipy.integrate as integrate

import plotly.graph_objects as go

In [5]:
## Useful constants
G = 6.6743e-11
m_earth = 5.97219e24
m_sun = 1.9891e30
au = 149597870700 # 1 Astronomical unit in meters
earth_linear_vel = 107550 # Taken from https://math.stackexchange.com/questions/2659117/what-is-the-linear-velocity-of-the-earth-in-kilometres-per-hour-please-help

In [6]:
## Two-body problem

earth_pos_0 = np.array([au, 0])
earth_vel_0 = np.array([earth_linear_vel, 0])

y_0 = np.concatenate([earth_pos_0, earth_vel_0])

def sun_earth_diffeq(t, y):
    dy = np.zeros(y.shape)
    
    earth_to_sun = -y[0:1]
    earth_to_sun = earth_to_sun / np.linalg.norm(earth_to_sun)
    
    force_sun_to_earth = G * m_sun * m_earth / au**2
    
    dy = earth_to_sun * force_sun_to_earth
    
    return dy